## Instalación de OpenMC

In [0]:
#
# Executing this cell you will install OpenMC and the nuclear
# data libraries in this instance of the Google Colaboratory virtual machine.
# The process takes about 2 minutes.
#

def install_openmc():
  #
  # Clone source code from Github, make and install
  #
  
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  
  %cd -q /content
  print("Obtaining OpenMC source code from Github...")
  !git  --no-pager clone https://github.com/mit-crpg/openmc.git &> /dev/null
  %cd -q openmc
  !git --no-pager checkout master &> /dev/null
  !mkdir build
  %cd -q build
  print("Running cmake...")
  !cmake ../ -DPYTHON_EXECUTABLE=/usr/bin/python3 -DCMAKE_INSTALL_PREFIX=/usr/local &> /dev/null
  print("Running make...")
  !make -j &> /dev/null
  print("Running make install...")
  !make install &> /dev/null
  import sys
  sys.path.append('/usr/local/lib/python3.6/dist-packages/openmc-0.10.0-py3.6-linux-x86_64.egg')
  sys.path.append('/usr/local/lib/python3.6/site-packages/')
  
  !apt update
  !apt install python3-vtk7 imagemagick
  
  %cd -q /content

def install_data_from_onedrive():
  #
  # Download preprocessed HDF5 files from OneDrive (faster).
  #
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala los datos nucleares de OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  %cd -q /content
  print("Obtaining HDF5 files from OneDrive...")
  !wget -O nndc_hdf5.tar.gz "https://onedrive.live.com/download?cid=22422A8EEA2A85B9&resid=22422A8EEA2A85B9%21105&authkey=AHL6xwYFXDwEzkk" &> /dev/null
  print("Uncompressing...")
  !tar xzvf nndc_hdf5.tar.gz &> /dev/null
    
from time import time
t1 = time()
install_openmc()
install_data_from_onedrive()
t2 = time()
print("Installed OpenMC in {:.2f} minutes".format((t2-t1)/60.0))


# Ejemplo: Elemento combustible MTR

In [0]:
import openmc
import numpy as np
%pylab inline

Vamos a modelar un elemento combustible tipo MTR. La definición que vamos a utilizar es la del apéndice F del [TECDOC 233](https://www.iaea.org/publications/484/research-reactor-core-conversion-from-the-use-of-highly-enriched-uranium-fuels-guidebook) "Research Reactor Core Conversion from the Use of Highly Enriched Uranium Fuels: Guidebook".

Comenzamos definiendo los materiales. Utilizaremos las composiciones que corresponden al elemento de HEU:

In [0]:
enriquecimiento = 0.93 # gU235/gU
m_U235 = 280 # g
vol_plate = 0.063*6.3*60 # cm^3
n_plates = 23
rho_U235 = m_U235/(vol_plate*n_plates)
rho_U = rho_U235/enriquecimiento
w_U = 0.21 # gU/gmeat
rho_meat = rho_U/w_U

mat_agua = openmc.Material()
mat_agua.add_nuclide("H1", 2.0, "ao")
mat_agua.add_nuclide("O16", 1.0, "ao")
mat_agua.add_s_alpha_beta("c_H_in_H2O")
mat_agua.set_density("g/cm3", 1.0)

mat_graphite = openmc.Material()
mat_graphite.add_nuclide("C0", 1.0, "ao")
mat_graphite.add_s_alpha_beta("c_Graphite")
mat_graphite.set_density("g/cm3", 1.7)

mat_meat = openmc.Material()
mat_meat.add_nuclide("U235", enriquecimiento*w_U, "wo")
mat_meat.add_nuclide("U238", (1.0-enriquecimiento)*w_U, "wo")
mat_meat.add_nuclide("Al27", 1.0-w_U, "wo")
mat_meat.set_density("g/cm3", rho_meat)

mat_al = openmc.Material()
mat_al.add_nuclide("Al27", 1.0, "ao")
mat_al.set_density("g/cm3", 2.7)

mats = openmc.Materials([mat_agua, mat_meat, mat_al, mat_graphite])
mats.cross_sections = "/content/nndc_hdf5/cross_sections.xml"
mats.export_to_xml()

Luego modelamos la placa combustible, que va a ser la unidad que utilizaremos para definir el elemento combustible.

In [0]:
espesor_placa = 0.127 # cm
ancho_placa = 7.6-2.0* 0.475 # cm
espesor_meat = 0.051 # cm
ancho_meat = 6.3 # cm
longitud_activa = 60 # cm

pl1 = openmc.XPlane(x0 = -(7.6/2.0 ) )
pl2 = openmc.XPlane(x0 = -(7.6/2.0 - 0.475))
pl3 = openmc.XPlane(x0 = +(7.6/2.0 - 0.475))
pl4 = openmc.XPlane(x0 = +(7.6/2.0))

region_meat = openmc.model.get_rectangular_prism(ancho_meat, espesor_meat)
region_placa = openmc.model.get_rectangular_prism(ancho_placa, espesor_placa)
cell_meat  = openmc.Cell(region=region_meat, fill=mat_meat)
cell_placa = openmc.Cell(region=region_placa & ~region_meat, fill=mat_al)
cell_agua1 = openmc.Cell(region=-pl1, fill=mat_agua)
cell_al1   = openmc.Cell(region=+pl1 & -pl2, fill=mat_al)
cell_agua2 = openmc.Cell(region=+pl2 & -pl3 & ~region_placa, fill=mat_agua)
cell_al2   = openmc.Cell(region=+pl3 & -pl4, fill=mat_al)
cell_agua3 = openmc.Cell(region=+pl4, fill=mat_agua)

uni_placa = openmc.Universe(cells=[cell_meat, cell_placa, cell_agua1, cell_al1,\
                              cell_agua2, cell_al2, cell_agua3])

El universo que corresponde a la placa se extiende hasta el infinito, lo que nos permite definir la separación entre placas (el *pitch*) en forma independiente:

In [0]:
uni_placa.plot(width=(8,1), color_by='material', colors={mat_meat:"Red", mat_al:"Gray", mat_agua:"Blue"})

Vamos a definir también un universo con una celda uniforme de agua, que nos va a servir para definir el exterior del elemento combustible:

In [0]:
cell_agua = openmc.Cell(fill=mat_agua)
uni_agua = openmc.Universe(cells=[cell_agua])

El elemento combustible es entonces un arreglo de 23 placas combustibles, rodeadas de agua:

In [0]:
lat1 = openmc.RectLattice()
lat1.lower_left = (-7.6/2.0,-8.05/2.0)
lat1.pitch = (7.6, 8.05/23.0)
lat1.outer = uni_agua
lat1.universes = [[uni_placa] for i in range(23)]

cell_nfe = openmc.Cell(fill=lat1)
uni_nfe = openmc.Universe(cells=[cell_nfe])
uni_nfe.plot(width=(10,10), color_by='material', colors={mat_meat:"Red", mat_al:"Gray", mat_agua:"Blue"})

Para realizar un cálculo de celda, vamos a aplicar este universo a una celda con condiciones de contorno reflejadas en $x$ y en $y$:

In [0]:
pl11 = openmc.XPlane(x0=-7.7/2.0, boundary_type="reflective")
pl12 = openmc.XPlane(x0=+7.7/2.0, boundary_type="reflective")
pl21 = openmc.YPlane(y0=-8.1/2.0, boundary_type="reflective")
pl22 = openmc.YPlane(y0=+8.1/2.0, boundary_type="reflective")

cell = openmc.Cell(region=+pl11&-pl12&+pl21&-pl22, fill=uni_nfe)

uni_root = openmc.Universe(cells=[cell])

geom = openmc.Geometry(uni_root)
geom.export_to_xml()

Llamamos al graficador:

In [0]:
plot2D = openmc.Plot()
plot2D.origin = (0, 0, 0)
plot2D.width = (7.7,8.1)
plot2D.pixels = (500, 500)
plot2D.id = 1
plot2D.colors={mat_meat:"Red", mat_al:"Gray", mat_agua:"Blue", mat_graphite:"Black"}
plot2D.color_by='material'
openmc.plot_inline(plots=[plot2D])

Y corremos el cálculo de celda:

In [0]:
run = openmc.Settings()
run.run_mode = "eigenvalue"
run.particles = 1000
run.batches = 110
run.inactive = 10
run.export_to_xml()

In [0]:
!rm summary.h5
!rm statepoint.*.h5
openmc.run()

In [0]:
estado = openmc.StatePoint("statepoint.110.h5")
keff = estado.k_generation
plot(keff)

In [0]:
a = hist(keff,bins=20)